MLDL_Tiny_Visual_Wake_Words


 Please install the required libraries

In [ ]:
!pip install wget
!pip install fvcore
!pip install pyvww
!pip install timm

  Preparing metadata (setup.py) ... done
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9657 sha256=5532454a667b1806b3e8c00ef1a97a80f56ae71aa3a2d2c62df7e0a078fd10b4
  Stored in directory: /root/.cache/pip/wheels/8b/f1/7f/5c94f0a7a505ca1c81cd1d9208ae2064675d97582078e6c769
Successfully built wget
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 2.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 5.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for fvcore: filename=fvcore-0.1.5.post20221221-py3-none-any.whl size=61405 sha256=50c332545ba4c5575e9c14b03542f6805a3a506e60d6d905235f8f592ba38ae2
  Stored in directory: /root/.cache/pip/wheels/01/c0/af/77c1cf53a1be9e42a52b48e5af2169d40ec2e89f7362489dd0
  Created wheel for iopath: filename=iopath-0.1.10-py3-none-any.whl size=31531 sha256=6fb652ccc5d939d1a7797b3ce5a28722b9d11ecdd973fc0891eefa7454ea5031
  Stored in directory:

**Dataset Loading**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
!python /content/drive/MyDrive/MLDL_Tiny_Visual_Wake_Words/data/download_coco_data.py

downloading train2017.zip ...
Downloading: 100% [19336861798 / 19336861798] bytes
Unzipping train2017.zip ...
downloading val2017.zip ...
Downloading: 100% [815585330 / 815585330] bytes
Unzipping val2017.zip ..
Moving all files into one folder ...
moved train files
moved val files
Download data complete


To create annotations

In [ ]:
TRAIN_ANNOTATIONS_FILE="COCOdataset/annotations/instances_train2017.json"
VAL_ANNOTATIONS_FILE="COCOdataset/annotations/instances_val2017.json"
DIR="COCOdataset/annotations/"
!python /content/drive/MyDrive/MLDL_Tiny_Visual_Wake_Words/data/create_coco_train_minival_split.py \
  --train_annotations_file="{TRAIN_ANNOTATIONS_FILE}" \
  --val_annotations_file="{VAL_ANNOTATIONS_FILE}" \
--output_dir="{DIR}"


In [ ]:
MAXITRAIN_ANNOTATIONS_FILE="COCOdataset/annotations/instances_maxitrain.json"
MINIVAL_ANNOTATIONS_FILE="COCOdataset/annotations/instances_minival.json"
VWW_OUTPUT_DIR="visualwakewords"
!python /content/drive/MyDrive/MLDL_Tiny_Visual_Wake_Words/data/create_visualwakewords_annotations.py \
  --train_annotations_file="{MAXITRAIN_ANNOTATIONS_FILE}" \
  --val_annotations_file="{MINIVAL_ANNOTATIONS_FILE}" \
  --output_dir="{VWW_OUTPUT_DIR}" \
  --threshold=0.005 \
  --foreground_class='person'

Processing /content/COCOdataset/annotations/instances_maxitrain.json...
loading annotations into memory...
Done (t=14.77s)
creating index...
index created!
There are 55233 images that now have label person, of the 115228 images in total.
Processing /content/COCOdataset/annotations/instances_minival.json...
loading annotations into memory...
Done (t=0.72s)
creating index...
index created!
There are 3800 images that now have label person, of the 8059 images in total.


# Various experiments



#### Execute the search algorithms



In [ ]:
!export 'PYTORCH_CUDA_ALLOC_CONF=max_split_size_mb:512'

Random search:

Perform the following steps with the provided parameters:

*   Run the search algorithm for a total of 20,000 iterations.
*   Evaluate the metrics without considering the cost, taking into account only Synflow and NASWOT.
*   Set the maximum number of blocks to 13, with a minimum of 3 blocks.
*   Allow for variable length of the model during the search process.







In [ ]:
!python /content/drive/MyDrive/MLDL_Tiny_Visual_Wake_Words/src/run_search.py \
    --algo random_search \
    --max_flops 200000000 \
    --max_params 2500000 \
    --metrics without_cost \
    --n_random 20000 \
    --max_blocks 13 \
    --save True

Start random search ...
100% 20000/20000 [30:00<00:00, 11.11it/s]
Finish random search.
Remaining 181 that satisfy constraints
Best exemplar obtained ---
Model: NetworkDecoded(
  (layers): Sequential(
    (0): InvertedResidualBlock(
      (conv1x1_1): Sequential(
        (0): Conv2d(3, 12, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (1): BatchNorm2d(12, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU(inplace=True)
      )
      (depthwise): Sequential(
        (0): Conv2d(12, 12, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), groups=12, bias=False)
        (1): BatchNorm2d(12, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU(inplace=True)
      )
      (conv1x1_2): Sequential(
        (0): Conv2d(12, 160, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (1): BatchNorm2d(160, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (relu): ReLU(inplace=True)
      (downsample): C

**Evolutionary algorithm**:
- Set the population size to 25.
- Enable crossover during the evolution process.
- Specify the number of generations as 1000.
- Evaluate the metrics without considering the cost.
- Allow for variable length of the models during the evolution.

In [ ]:
!python /content/drive/MyDrive/MLDL_Tiny_Visual_Wake_Words/src/run_search.py \
    --algo "ea_search" \
    --max_flops 200000000 \
    --max_params 2500000 \
    --initial_pop 25 \
    --generation_ea 1000 \
    --max_blocks 13 \
    --save True

In the random search algorithm, the evaluation of models is based on training-free metrics, as well as the number of parameters and flops. Models with lower computational cost, indicated by fewer parameters and flops, are given higher priority in the ranking compared to models with higher computational requirements.

## Training

**The model trained using the random search algorithm**

In [ ]:
!python /content/drive/MyDrive/MLDL_Tiny_Visual_Wake_Words/src/run_train.py \
    --model "Model_random_search.pth" \
    --root_data "COCOdataset/all2017" \
    --ann_train "visualwakewords/instances_train.json" \
    --ann_val "visualwakewords/instances_val.json" \
    --batch_size 256 \
    --learning_rate 0.01 \
    --momentum 0.9 \
    --epochs 40 \
    --weight_decay 0.0001


loading annotations into memory...
Done (t=3.88s)
creating index...
index created!
loading annotations into memory...
Done (t=0.59s)
creating index...
index created!
training epoch number 0.00 of total epochs of 40.00
100% 361/361 [08:33<00:00,  1.42s/it, training with Current loss: 0.0026, Accuracy: 58.2602, at iteration: 360.0]
100% 91/91 [02:01<00:00,  1.33s/it, validation with Current loss: 0.0026, Accuracy: 62.4821, at iteration: 90.0]
Epoch: 1
	 Training loss 0.00262, Training accuracy 58.26
	 Validation loss 0.00260, Validation accuracy 62.48
training epoch number 1.00 of total epochs of 40.00
100% 361/361 [08:37<00:00,  1.43s/it, training with Current loss: 0.0024, Accuracy: 66.1076, at iteration: 360.0]
100% 91/91 [02:00<00:00,  1.32s/it, validation with Current loss: 0.0024, Accuracy: 66.6826, at iteration: 90.0]
Epoch: 2
	 Training loss 0.00242, Training accuracy 66.11
	 Validation loss 0.00243, Validation accuracy 66.68
training epoch number 2.00 of total epochs of 40.00
10

**The model trained using the evolutionary algorithm**

In [ ]:
!python /content/drive/MyDrive/MLDL_Tiny_Visual_Wake_Words/src/run_train.py \
    --model "Model_ea_search.pth" \
    --root_data "COCOdataset/all2017" \
    --ann_train "visualwakewords/instances_train.json" \
    --ann_val "visualwakewords/instances_val.json" \
    --batch_size 256 \
    --learning_rate 0.1 \
    --momentum 0.9 \
    --epochs 40 \
    --weight_decay 0.0001


loading annotations into memory...
Done (t=4.01s)
creating index...
index created!
loading annotations into memory...
Done (t=0.22s)
creating index...
index created!
training epoch number 0.00 of total epochs of 40.00
100% 361/361 [04:07<00:00,  1.46it/s, training with Current loss: 0.0027, Accuracy: 53.6400, at iteration: 360.0]
100% 91/91 [01:03<00:00,  1.44it/s, validation with Current loss: 0.0027, Accuracy: 56.5112, at iteration: 90.0]
Epoch: 1
	 Training loss 0.00270, Training accuracy 53.64
	 Validation loss 0.00269, Validation accuracy 56.51
training epoch number 1.00 of total epochs of 40.00
100% 361/361 [04:06<00:00,  1.46it/s, training with Current loss: 0.0026, Accuracy: 58.7766, at iteration: 360.0]
100% 91/91 [01:03<00:00,  1.43it/s, validation with Current loss: 0.0026, Accuracy: 60.1606, at iteration: 90.0]
Epoch: 2
	 Training loss 0.00263, Training accuracy 58.78
	 Validation loss 0.00263, Validation accuracy 60.16
training epoch number 2.00 of total epochs of 40.00
10

In [ ]:
!python /content/drive/MyDrive/MLDL_Tiny_Visual_Wake_Words/src/run_search.py \
    --algo  MobileNetV3 \
    --save True

Unsupported operator aten::relu6_ encountered 28 time(s)
Unsupported operator aten::mul encountered 28 time(s)
Unsupported operator aten::div encountered 28 time(s)
MobileNetV3 --- 
 params = 218234 flops = 174919648 

Synflow score: 294823.3797965417
NASWOT score: 454.96044921875


**Training using MobileNet V3**

In [ ]:
!python /content/drive/MyDrive/MLDL_Tiny_Visual_Wake_Words/src/run_train.py \
    --model "/content/drive/MyDrive/MLDL_Tiny_Visual_Wake_Words/src/Model_MobileNetV3.pth" \
    --root_data "COCOdataset/all2017" \
    --ann_train "visualwakewords/instances_train.json" \
    --ann_val "visualwakewords/instances_val.json" \
    --batch_size 32 \
    --learning_rate 0.04 \
    --momentum 0.9 \
    --epochs 40 \
    --weight_decay 0.00001

loading annotations into memory...
Done (t=4.14s)
creating index...
index created!
loading annotations into memory...
Done (t=0.21s)
creating index...
index created!
training epoch number 0.00 of total epochs of 40.00
100% 2881/2881 [08:47<00:00,  5.46it/s, training with Current loss: 0.0200, Accuracy: 62.8359, at iteration: 2880.0]
100% 721/721 [02:01<00:00,  5.95it/s, validation with Current loss: 0.0189, Accuracy: 66.9473, at iteration: 720.0]
Epoch: 1
	 Training loss 0.02002, Training accuracy 62.84
	 Validation loss 0.01891, Validation accuracy 66.95
training epoch number 1.00 of total epochs of 40.00
100% 2881/2881 [08:52<00:00,  5.41it/s, training with Current loss: 0.0181, Accuracy: 69.7178, at iteration: 2880.0]
100% 721/721 [02:02<00:00,  5.89it/s, validation with Current loss: 0.0172, Accuracy: 71.2562, at iteration: 720.0]
Epoch: 2
	 Training loss 0.01807, Training accuracy 69.72
	 Validation loss 0.01723, Validation accuracy 71.26
training epoch number 2.00 of total epochs